# Evaluation using RAGAS

RAGAS (Retrieval-Augmented Generation Assessment Suite) is an open-source framework designed to systematically evaluate retrieval-augmented generation (RAG) pipelines. It measures key aspects of performance including faithfulness (factual grounding), answer relevancy, context precision, context recall, and overall answer correctness. By combining LLM-based evaluations with traditional retrieval metrics, RAGAS provides a structured and quantitative way to assess how well a RAG system retrieves relevant information and generates accurate, context-based answers.

In [16]:
from datasets import Dataset

from ragas.metrics import (
    faithfulness,
    answer_relevancy,
    context_precision,
    context_recall,
    answer_correctness,
)
from ragas import evaluate

try:
    from dotenv import load_dotenv
    load_dotenv()
except ImportError:
    pass  # dotenv not used in deployment

import os

import pandas as pd

In [10]:
# gets open ai key
openai_api = os.getenv("OPENAI_API_KEY")

# this tells RAGAS / LangChain / OpenAI Python SDK to use the key
# it uses gpt-3.5-turbo by default
os.environ["OPENAI_API_KEY"] = openai_api

### Evaluation using simple chunking

In [11]:
# each question is passed through the RAG model and the answer is what was generated
data = [
    {
        "question": "What is Reform UK's policy on immigration?",
        "answer": "Reform UK's policy on immigration includes freezing non-essential immigration, stopping illegal immigrants with a 4 Point Plan, securing detention for all illegal migrants, immediate deportation for foreign criminals, barring student dependents, stopping health tourism and immediate access to benefits, and implementing an employer immigration tax.",
        "contexts": [
            "Reform UK proposes to freeze non-essential immigration, stop the small boats, and deport illegal migrants. They also plan to impose a five-year residency requirement for claiming benefits."
        ],
        "ground_truth": "Freeze non-essential immigration, stop small boats, deport illegal migrants, five-year residency rule for benefits."
    },
    {
        "question": "How do the Liberal Democrats plan to improve access to GPs?",
        "answer": """The Liberal Democrats plan to improve access to GPs by giving everyone the right to see a GP within seven days, or within 24 hours if they urgently need to. They also aim to achieve this by increasing the number of full-time equivalent GPs by 8,000, half through boosting recruitment and half from retaining more experienced GPs. Additionally, they plan to free up GPs' time by giving more prescribing rights and public health advisory services to qualified pharmacists, nurse practitioners, and paramedics, as well as establishing a universal 
        24/7 GP booking system and removing top-down bureaucracy to allow practices to hire the staff they need and invest in training.""",
        "contexts": [
            "Liberal Democrats will give everyone a new right to see a GP within seven days, or 24 hours if urgent, with extra doctors to support this plan."
        ],
        "ground_truth": "Right to see a GP within seven days or 24 hours if urgent, supported by hiring more doctors."
    },
    {
        "question": "What are Labour's five national missions according to the 2024 manifesto?",
        "answer": """  1) Kickstart economic growth to secure the highest sustained growth in the G7
                        2) Make Britain a clean energy superpower
                        3) Take back our streets by halving serious violent crime
                        4) Break down barriers to opportunity
                        5) Build an NHS fit for the future""",
        "contexts": [
            "Labour’s five missions are: kickstart economic growth, make Britain a clean energy superpower, take back our streets, break down barriers to opportunity, and build an NHS fit for the future."
        ],
        "ground_truth": "Kickstart economic growth, clean energy superpower, take back streets, break barriers to opportunity, build a future-ready NHS."
    },
    {
        "question": "What is the Green Party's position on NHS funding?",
        "answer": """The Green Party's position on NHS funding is to increase taxes on the super-rich to fund the national health service properly, fix the broken care system that is causing problems for the NHS, and ensure a fully public health service with no profit motive involved. They estimate that additional annual expenditure of £8bn in the first full year of the next parliament, rising to a total of £28bn in 2030, is needed for the NHS in England. They also estimate that additional capital spending of at least £20bn is needed over the next five years. Additionally, the Green Party plans to invest £20bn in NHS budgets over the life of the parliament for hospital building and repair, and to reduce hospital waiting lists by giving Hospital Trusts long-term funding commitments.""",
        "contexts": [
            "The Green Party pledges to increase NHS spending by £8 billion in the first year, rising to £28 billion by 2030, and keep the NHS fully public."
        ],
        "ground_truth": "Increase NHS spending significantly and ensure the NHS remains fully public."
    },
    {
        "question": "How do the Conservatives plan to reduce immigration?",
        "answer": """The Conservatives plan to reduce immigration by introducing a binding, legal cap on migration, setting it on work and family visas to protect public services while bringing in 
                     the necessary skills for businesses and the NHS. They will set the cap at a level that considers the costs and benefits of migration, with the cap decreasing every year and requiring an annual vote in Parliament to control immigration numbers. They also aim to deter illegal migration by implementing measures such as removing the incentive to come illegally, running 
                     a process of permanently removing illegal migrants to Rwanda, cracking down on organised immigration crime, and reforming asylum rules.""",
        "contexts": [
            "The Conservatives plan to introduce a legal cap on migration numbers, remove illegal migrants to Rwanda, and rewrite asylum treaties."
        ],
        "ground_truth": "Legal migration cap, Rwanda deportation policy, rewrite asylum treaties."
    },
    {
        "question": "What is Reform UK's policy on NHS waiting lists?",
        "answer": "Reform UK's policy on NHS waiting lists includes critical reforms needed in the first 100 days such as ending doctor and nurse shortages, utilizing independent healthcare capacity, providing tax relief on private healthcare and insurance, and implementing a new NHS voucher scheme for patients who can't see a GP within a certain timeframe. They also aim to improve efficiency, cut waste, and unnecessary managers in the NHS to reduce waiting times.",
        "contexts": [
            "Reform UK pledges to achieve zero NHS waiting lists by reforming healthcare delivery, cutting back-office waste, and offering tax breaks to doctors and nurses to tackle staffing shortages."
        ],
        "ground_truth": "Zero NHS waiting lists through healthcare reform, cutting waste, and tax breaks for staff."
    },
    {
        "question": "How will the Liberal Democrats tackle climate change?",
        "answer": """The Liberal Democrats plan to tackle climate change by putting it at the heart of a new industrial strategy. They will cut emissions and bills with an emergency Home Energy Upgrade programme, drive a rooftop solar revolution, and invest in clean energy, transport, and industry. They also aim to restore nature, tackle toxic air pollution, and provide skills training, incentives, and advice to help families and businesses transition to net zero. Additionally, they plan to regulate financial services to encourage climate-friendly investments, support British industry to cut emissions, and provide more advice to companies on cutting emissions.""",
        "contexts": [
            "The Liberal Democrats will drive a rooftop solar revolution, invest in clean energy and transport, ban water companies from dumping sewage, and implement an emergency Home Energy Upgrade programme."
        ],
        "ground_truth": "Invest in clean energy, rooftop solar, ban sewage dumping, and upgrade home energy."
    },
    {
        "question": "What energy policy is proposed by Labour in their 2024 manifesto?",
        "answer": """Labour proposes to set up Great British Energy to cut bills for good, achieve energy independence from dictators like Putin, create 650,000 new high-quality jobs, warm homes to slash fuel poverty, and force water companies to clean up rivers. They also plan to deploy more distributed production capacity through their Local Power Plan, partnering with energy 
                     companies, local authorities, and co-operatives to install clean power projects such as onshore wind, solar, and hydropower projects. They aim to ensure local people benefit directly from this energy production and reform the energy system to put consumers first and attract the investment needed to cut bills.""",
        "contexts": [
            "Labour proposes creating a publicly-owned clean power company, Great British Energy, to cut bills, boost energy security, and achieve zero-carbon electricity by 2030."
        ],
        "ground_truth": "Create Great British Energy, deliver zero-carbon electricity by 2030."
    },
    {
        "question": "What is the Green Party’s approach to public transport?",
        "answer": """The Green Party's approach to public transport includes championing public transport for every community to thrive, increasing annual public subsidies for rail and bus travel, 
                     investing in new cycleways and footpaths, bringing the railways back into public ownership, giving local authorities control over and funding for improved bus services, and ensuring investment in new cycleways and footpaths. They also aim to increase public transport subsidies to make it reliable, frequent, accessible, and affordable, including providing free bus travel for under-18s. Additionally, they oppose all new road building plans and aim to shift the transport system away from cars and roads by promoting the use of electric vehicles and active travel options like walking and cycling.""",
        "contexts": [
            "The Green Party plans to invest in public transport, including rail, bus, and active travel infrastructure, to reduce car dependency and cut emissions."
        ],
        "ground_truth": "Invest in rail, bus, and active travel infrastructure to cut emissions and car dependency."
    },
    {
        "question": "How do the Conservatives propose to support first-time home buyers?",
        "answer": """The Conservatives propose to support first-time home buyers by ensuring the majority of first-time buyers pay no Stamp Duty at all, making permanent the increase to the threshold at which first-time buyers pay Stamp Duty to £425,000, launching a new and improved Help to Buy scheme to provide first-time buyers with an equity loan of up to 20% towards the cost of a new build home, and continuing the Mortgage Guarantee Scheme. They also plan to legislate for new 'Local Connection' and 'UK Connection' tests for social housing in England, implement a 'three strikes and you're out' expectation for social housing landlords for anti-social behavior, and protect family homes from higher tax.""",
        "contexts": [
            "The Conservatives propose abolishing Stamp Duty for homes up to £425,000 for first-time buyers and introducing a new Help to Buy scheme."
        ],
        "ground_truth": "Abolish Stamp Duty for homes up to £425,000 and introduce Help to Buy scheme."
    }
]

In [12]:
# creates hugging face dataset
ragas_dataset = Dataset.from_list(data)

In [13]:
# sets list of metrics to evaluate by
metrics = [
    faithfulness,
    answer_relevancy,
    context_precision,
    context_recall,
    answer_correctness
]

### Results

In [14]:
results = evaluate(ragas_dataset, metrics=metrics)
print(results)

Evaluating: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 50/50 [00:38<00:00,  1.31it/s]


{'faithfulness': 0.2373, 'answer_relevancy': 0.9536, 'context_precision': 1.0000, 'context_recall': 1.0000, 'answer_correctness': 0.5125}


In [28]:
# creates dataframe of all results for each question
scores_per_sample = pd.DataFrame(results.scores)

# 1. Show per-sample scores
scores_per_sample

,faithfulness,answer_relevancy,context_precision,context_recall,answer_correctness
0,0.142857,0.957125,1.0,1.0,0.600007
1,0.166667,0.991475,1.0,1.0,0.439828
2,0.600000,0.866082,1.0,1.0,0.985265
3,0.375000,1.000000,1.0,1.0,0.342492
4,0.363636,0.953771,1.0,1.0,0.317546
5,0.250000,0.948528,1.0,1.0,0.333163
6,0.100000,0.973649,1.0,1.0,0.508517
7,0.000000,0.904589,1.0,1.0,0.330308
8,0.250000,0.997762,1.0,1.0,0.464498
9,0.125000,0.943372,1.0,1.0,0.803018


In [29]:
# 2. Calculate average across all samples
mean_scores = scores_per_sample.mean()
print("\nAverage Scores:")
print(mean_scores)


Average Scores:
faithfulness          0.237316
answer_relevancy      0.953635
context_precision     1.000000
context_recall        1.000000
answer_correctness    0.512464
dtype: float64
